In [1]:
import hd_features as hd
import hd_analysis as hda
import pandas as pd
from bokeh.models import Panel, Tabs
from bokeh.plotting import show,save,output_file

In [2]:
##multiple timestamp calulation for given time range ###################################
start_date = 1781-30,1,1,1,1
end_date = 2027+30, 12, 31,23,1
percentage = 1 # 1-> 100%,
time_unit = "days" #precision of timestamps. Years,days,months,hours are supported for analyis
intervall = 1 #every X unit
num_cpu = 6

result,timestamp_list = hd.calc_mult_hd_features(start_date,end_date,percentage,time_unit,intervall,num_cpu)
result_lists = hd.unpack_mult_features(result,full=True) #lists are structured as dict
Map_fct = pd.Series([1]*len(timestamp_list))#use this if no mapping function shall apply 


  0%|          | 0/112129 [00:00<?, ?it/s]

In [3]:
###multiple timestamp calulation for known birth rates#########################

##for comparison with unknown (constant) birth rate distribution use distribution=False
#Map_fct,timestamp_list = hda.get_map_fct("birth_count_from_1969_2014.csv",distribution=True)
#result_lists = hda.calc_mult_hd_features_birth_dist(timestamp_list,num_cpu=6)
##################################################################

In [4]:
%%time
order="sorted" # "sorted" ->numeric order, "rave" -> IGING circle number order
norm=False #if normalisation of timeseries is needed set True?
pie_fig_size = [400,350] #size of pie charts
#if not all features shall be calculated, remove them here
ohe_fig_size_dict = {
                     "typ":[1200,550],
                     "auth":[1200,550],
                     "profile":[1200,550],
                     "split":[1200,550],
                     "active_chakra":[1200,550],
                     "gate":[1200,1700],
                     "active_channel":[1200,1000],
                     #"inc_cross":[1200,2500],
                     "inc_cross_typ":[1200,550]
                    }
hd_feature_list = list(ohe_fig_size_dict.keys())

"""
time_intervall -> grouping intervall for figures x-Achses
years, months,days,hours, continous, custom XX are supported
for custom avail. frequencies see offset aliases pd.grouper function (e.g custom 1W)
"""
time_intervall = "years" 
print("ohe_encoding")
ohe_data_dict = hda.ohe_hd_features(result_lists,
                                    hd_feature_list,
                                    timestamp_list,
                                    Map_fct,
                                    time_intervall,
                                    norm=norm,
                                    order=order)
#display graphs
print("build figures")
stat_graph = hda.get_hd_stat_graph(result_lists,ohe_data_dict,hd_feature_list,pie_fig_size,Map_fct) #basic statistics
tab_dict = hda.get_tabbed_hd_ts(ohe_data_dict,time_intervall,hd_feature_list,ohe_fig_size_dict) #timerseries

tab_dict["stat_graph_tab"] = Panel(child = stat_graph, title = "Basic Statistics")
tab_keys = ["stat_graph"] + list(ohe_fig_size_dict.keys())
tabs = Tabs(tabs=[tab_dict["{}_tab".format(elem)] 
                  for elem in tab_keys])
show(tabs)

ohe_encoding


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.17it/s]


build figures


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 19.98it/s]


Wall time: 8.61 s


In [5]:
#output_file(filename="hd_analysis_per_day_unknown_dist.html", title="hd_analysis_per_day_unknown_dist")
#save(tabs)